# Imports

In [1]:
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import torchvision

from argparse import ArgumentParser   
from scipy import interpolate
from scipy.ndimage import gaussian_filter

from hcpinnseikonal.utils import *
from hcpinnseikonal.model import *
from hcpinnseikonal.train3dwell import *
from hcpinnseikonal.plot import *
from hcpinnseikonal.arguments import *

args = parser.parse_args([])

args.use_wandb='n'
args.project_name='GFATT_PINNs-21-3d-field'

dict_args = vars(args)
print(dict_args)

# Change these lines for the wandb setup
if args.use_wandb=='y':
    wandb.init(project=args.project_name)
    wandb.run.log_code(".")
    wandb_dir = wandb.run.dir
else:
    args.save_folder='../saves/saves_randomPINNs'
    from pathlib import Path
    Path(args.save_folder).mkdir(parents=True, exist_ok=True)
    wandb_dir = args.save_folder
    
# plt.style.use("~/science.mplstyle")

{'lateral_spacing': 0.01, 'vertical_spacing': 0.01, 'max_offset': 5.0, 'max_depth': 1.0, 'rec_spacing': 10, 'sou_spacing': 10, 'num_epochs': 250, 'num_neurons': 20, 'num_layers': 10, 'learning_rate': 0.001, 'model_type': 'seam', 'data_type': 'full', 'middle_shot': 'n', 'until_cmb': 'n', 'earth_scale': 'n', 'scale_factor': 10, 'reduce_after': 15, 'seed': 123, 'initialization': 'varianceScaling', 'plotting_factor': 1, 'rescale_plot': 'n', 'depth_shift': 'n', 'tau_multiplier': 3.0, 'initial_velocity': 4, 'zid_source': 5, 'zid_receiver': 0, 'explode_reflector': 'n', 'field_synthetic': 'n', 'v_multiplier': 3, 'activation': 'elu', 'num_points': 1.0, 'irregular_grid': 'n', 'xid_well': 5, 'last_vmultiplier': 5, 'v_units': 'unitless', 'well_depth': None, 'exp_function': 'n', 'exp_factor': 1.0, 'exclude_topo': 'n', 'exclude_well': 'n', 'exclude_source': 'n', 'loss_function': 'mse', 'station_factor': 1.0, 'event_factor': 1.0, 'checker_size': 5.0, 'tau_act': 'None', 'empty_middle': 'n', 'factoriza

# Random Field

In [2]:
import torch
import math

from timeit import default_timer


class GaussianRandomField(object):

    def __init__(self, dim, size, alpha=2, tau=3, sigma=None, boundary="periodic", device=None):

        self.dim = dim
        self.device = device

        if sigma is None:
            sigma = tau**(0.5*(2*alpha - self.dim))

        k_max = size//2

        if dim == 1:
            k = torch.cat((torch.arange(start=0, end=k_max, step=1, device=device), \
                           torch.arange(start=-k_max, end=0, step=1, device=device)), 0)

            self.sqrt_eig = size*math.sqrt(2.0)*sigma*((4*(math.pi**2)*(k**2) + tau**2)**(-alpha/2.0))
            self.sqrt_eig[0] = 0.0

        elif dim == 2:
            wavenumers = torch.cat((torch.arange(start=0, end=k_max, step=1, device=device), \
                                    torch.arange(start=-k_max, end=0, step=1, device=device)), 0).repeat(size,1)

            k_x = wavenumers.transpose(0,1)
            k_y = wavenumers

            self.sqrt_eig = (size**2)*math.sqrt(2.0)*sigma*((4*(math.pi**2)*(k_x**2 + k_y**2) + tau**2)**(-alpha/2.0))
            self.sqrt_eig[0,0] = 0.0

        elif dim == 3:
            wavenumers = torch.cat((torch.arange(start=0, end=k_max, step=1, device=device), \
                                    torch.arange(start=-k_max, end=0, step=1, device=device)), 0).repeat(size,size,1)

            k_x = wavenumers.transpose(1,2)
            k_y = wavenumers
            k_z = wavenumers.transpose(0,2)

            self.sqrt_eig = (size**3)*math.sqrt(2.0)*sigma*((4*(math.pi**2)*(k_x**2 + k_y**2 + k_z**2) + tau**2)**(-alpha/2.0))
            self.sqrt_eig[0,0,0] = 0.0

        self.size = []
        for j in range(self.dim):
            self.size.append(size)

        self.size = tuple(self.size)

    def sample(self, N):

        coeff = torch.randn(N, *self.size, dtype=torch.cfloat, device=self.device)
        coeff = self.sqrt_eig * coeff

        return torch.fft.ifftn(coeff, dim=list(range(-1, -self.dim - 1, -1))).real

In [3]:
# Resolution
s = 256

# Number of solutions to generate
N = 500

#Set up 2d GRF with covariance parameters
random_field = GaussianRandomField(2, s, alpha=2.5, tau=7, device='cuda')

# Sample random feilds
samples = random_field.sample(N) + 5
 
# Plot
for i in range(N//100):
    plot_section(samples[i,:,:].detach().cpu().numpy(), 'samples_'+str(i)+'_.pdf', 
                 save_dir='./', aspect='auto')

RuntimeError: CUDA out of memory. Tried to allocate 250.00 MiB (GPU 0; 47.45 GiB total capacity; 250.25 MiB already allocated; 79.25 MiB free; 252.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Dataset

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = target
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [ ]:
C, H, W = 1, 256, 256

transform = torchvision.transforms.Compose([torchvision.transforms.ToPILImage(), torchvision.transforms.ToTensor()])

batch_size = 64

train_dataset = MyDataset(samples[:400,:,:].reshape(-1, 1, H, W,), samples[:400,:,:].reshape(-1, 1, H, W,), transform)
val_dataset = MyDataset(samples[400:,:,:].reshape(-1, 1, H, W,), samples[400:,:,:].reshape(-1, 1, H, W,), transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

# UNet

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(
        self,
        in_channels=1,
        n_classes=2,
        depth=5,
        wf=6,
        padding=False,
        batch_norm=False,
        up_mode='upconv',
    ):
        """
        Implementation of
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        (Ronneberger et al., 2015)
        https://arxiv.org/abs/1505.04597
        Using the default arguments will yield the exact version used
        in the original paper
        Args:
            in_channels (int): number of input channels
            n_classes (int): number of output channels
            depth (int): depth of the network
            wf (int): number of filters in the first layer is 2**wf
            padding (bool): if True, apply padding such that the input shape
                            is the same as the output.
                            This may introduce artifacts
            batch_norm (bool): Use BatchNorm after layers with an
                               activation function
            up_mode (str): one of 'upconv' or 'upsample'.
                           'upconv' will use transposed convolutions for
                           learned upsampling.
                           'upsample' will use bilinear upsampling.
        """
        super(UNet, self).__init__()
        assert up_mode in ('upconv', 'upsample')
        self.padding = padding
        self.depth = depth
        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(
                UNetConvBlock(prev_channels, 2 ** (wf + i), padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.up_path = nn.ModuleList()
        for i in reversed(range(depth - 1)):
            self.up_path.append(
                UNetUpBlock(prev_channels, 2 ** (wf + i), up_mode, padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1)

    def forward(self, x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i != len(self.down_path) - 1:
                blocks.append(x)
                x = F.max_pool2d(x, 2)

        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i - 1])

        output = self.last(x)

        return output


class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, padding, batch_norm):
        super(UNetConvBlock, self).__init__()
        block = []

        block.append(nn.Conv2d(in_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        block.append(nn.Conv2d(out_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        block.append(nn.Dropout2d(p=0.15)) # edited
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        out = self.block(x)
        return out


class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, up_mode, padding, batch_norm):
        super(UNetUpBlock, self).__init__()
        if up_mode == 'upconv':
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2, stride=2)
        elif up_mode == 'upsample':
            self.up = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_size, out_size, kernel_size=1),
            )

        self.conv_block = UNetConvBlock(in_size, out_size, padding, batch_norm)

    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[
            :, :, diff_y : (diff_y + target_size[0]), diff_x : (diff_x + target_size[1])
        ]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)

        return out

## Train

In [ ]:
import sys, os, time, glob, time, pdb
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('device: ', device)

script_time = time.time()

def plot_losses(running_train_loss, running_val_loss, train_epoch_loss, val_epoch_loss, epoch):
    fig = plt.figure(figsize=(16,16))
    fig.suptitle('loss trends', fontsize=20)
    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)

    ax1.title.set_text('epoch train loss VS #epochs')
    ax1.set_xlabel('#epochs')
    ax1.set_ylabel('epoch train loss')
    ax1.plot(train_epoch_loss)
    
    ax2.title.set_text('epoch val loss VS #epochs')
    ax2.set_xlabel('#epochs')
    ax2.set_ylabel('epoch val loss')
    ax2.plot(val_epoch_loss)
 
    ax3.title.set_text('batch train loss VS #batches')
    ax3.set_xlabel('#batches')
    ax3.set_ylabel('batch train loss')
    ax3.plot(running_train_loss)

    ax4.title.set_text('batch val loss VS #batches')
    ax4.set_xlabel('#batches')
    ax4.set_ylabel('batch val loss')
    ax4.plot(running_val_loss)
    
    plt.savefig(os.path.join('./','losses_{}.png'.format(str(epoch + 1).zfill(2))))

# defining the model
model = UNet(n_classes = 1, depth = 4, padding = True).to(device) # try decreasing the depth value if there is a memory error

train_epoch_loss = []
val_epoch_loss = []
running_train_loss = []
running_val_loss = []
epochs_till_now = 0

lr = 1e-5
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = lr)
loss_fn = nn.MSELoss()

log_interval = 20
epochs = 100

for epoch in range(epochs_till_now, epochs_till_now+epochs):
    print('\n===== EPOCH {}/{} ====='.format(epoch + 1, epochs_till_now + epochs))    
    print('\nTRAINING...')
    epoch_train_start_time = time.time()
    model.train()
    for batch_idx, (imgs, noisy_imgs) in enumerate(train_loader):
        batch_start_time = time.time()
        imgs = imgs.to(device)
        noisy_imgs = noisy_imgs.to(device)

        optimizer.zero_grad()
        out = model(noisy_imgs)

        loss = loss_fn(out, imgs)
        running_train_loss.append(loss.item())
        loss.backward()
        optimizer.step()

        if (batch_idx + 1)%log_interval == 0:
            batch_time = time.time() - batch_start_time
            m,s = divmod(batch_time, 60)
            print('train loss @batch_idx {}/{}: {} in {} mins {} secs (per batch)'.format(str(batch_idx+1).zfill(len(str(len(train_loader)))), len(train_loader), loss.item(), int(m), round(s, 2)))

    train_epoch_loss.append(np.array(running_train_loss).mean())

    epoch_train_time = time.time() - epoch_train_start_time
    m,s = divmod(epoch_train_time, 60)
    h,m = divmod(m, 60)
    print('\nepoch train time: {} hrs {} mins {} secs'.format(int(h), int(m), int(s)))

    print('\nVALIDATION...')
    epoch_val_start_time = time.time()
    model.eval()
    with torch.no_grad():
        for batch_idx, (imgs, noisy_imgs) in enumerate(val_loader):

            imgs = imgs.to(device)
            noisy_imgs = noisy_imgs.to(device)

            out = model(noisy_imgs)
            loss = loss_fn(out, imgs)

            running_val_loss.append(loss.item())

            if (batch_idx + 1)%log_interval == 0:
                print('val loss   @batch_idx {}/{}: {}'.format(str(batch_idx+1).zfill(len(str(len(val_loader)))), len(val_loader), loss.item()))

    val_epoch_loss.append(np.array(running_val_loss).mean())

    epoch_val_time = time.time() - epoch_val_start_time
    m,s = divmod(epoch_val_time, 60)
    h,m = divmod(m, 60)
    print('\nepoch val   time: {} hrs {} mins {} secs'.format(int(h), int(m), int(s)))

    plot_losses(running_train_loss, running_val_loss, train_epoch_loss, val_epoch_loss,  epoch)   

    torch.save({'model_state_dict': model.state_dict(), 
                'losses': {'running_train_loss': running_train_loss, 
                           'running_val_loss': running_val_loss, 
                           'train_epoch_loss': train_epoch_loss, 
                           'val_epoch_loss': val_epoch_loss}, 
                'epochs_till_now': epoch+1}, 
                os.path.join('./', 'model{}.pth'.format(str(epoch + 1).zfill(2))))

total_script_time = time.time() - script_time
m, s = divmod(total_script_time, 60)
h, m = divmod(m, 60)
print(f'\ntotal time taken for running this script: {int(h)} hrs {int(m)} mins {int(s)} secs')
  
print('\nFin.')

# AutoEncoder

In [ ]:
def make_MLP(din, dout, hidden=None, nonlin=nn.ELU, output_nonlin=None, bias=True, output_bias=None):
    '''
    :param din: int
    :param dout: int
    :param hidden: ordered list of int - each element corresponds to a FC layer with that width (empty means network is not deep)
    :param nonlin: str - choose from options found in get_nonlinearity(), applied after each intermediate layer
    :param output_nonlin: str - nonlinearity to be applied after the last (output) layer
    :return: an nn.Sequential instance with the corresponding layers
    '''

    if hidden is None:
        hidden = []

    if output_bias is None:
        output_bias = bias

    flatten = False
    reshape = None

    if isinstance(din, (tuple, list)):
        flatten = True
        din = int(np.product(din))
    if isinstance(dout, (tuple, list)):
        reshape = dout
        dout = int(np.product(dout))

    nonlins = [nonlin] * len(hidden) + [output_nonlin]
    biases = [bias] * len(hidden) + [output_bias]
    hidden = din, *hidden, dout

    layers = []
    if flatten:
        layers.append(nn.Flatten())

    for in_dim, out_dim, nonlin, bias in zip(hidden, hidden[1:], nonlins, biases):
        layer = nn.Linear(in_dim, out_dim, bias=bias)
        layers.append(layer)
        if nonlin is not None:
            layers.append(nonlin())

    if reshape is not None:
        layers.append(Reshaper(reshape))


    net = nn.Sequential(*layers)

    net.din, net.dout = din, dout
    return net

class ConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int = 3, stride: int = 1, padding: int = 1,
                 pool: Optional[str] = None, unpool: Optional[str] = None, pool_size: int = 2,
                 nonlin: Optional[Type[nn.Module]] = nn.ELU, norm: Optional[str] = 'batch', **kwargs):
        super().__init__()
        self.unpool = None if unpool is None else nn.Upsample(scale_factor=pool_size, mode=unpool)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, **kwargs)
        self.pool = None if pool is None \
            else (nn.MaxPool2d(pool_size) if pool == 'max' else nn.AvgPool2d(pool_size))
        if norm == 'batch':
            self.norm = nn.BatchNorm2d(out_channels)
        elif norm == 'instance':
            self.norm = nn.InstanceNorm2d(out_channels)
        elif norm == 'group':
            self.norm = nn.GroupNorm(8, out_channels)
        else:
            self.norm = None
        self.nonlin = nonlin()


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        c = self.unpool(x) if self.unpool is not None else x
        c = self.conv(c)
        if self.pool is not None:
            c = self.pool(c)
        if self.norm is not None:
            c = self.norm(c)
        if self.nonlin is not None:
            c = self.nonlin(c)
        return c

class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            ConvBlock( 3, 64, 5, 1, 2, pool='max', norm='group', nonlin=nn.ELU),
            ConvBlock(64, 64, 3, 1, 1, pool='max', norm='group', nonlin=nn.ELU),
            ConvBlock(64, 64, 3, 1, 1, pool='max', norm='group', nonlin=nn.ELU),
            ConvBlock(64, 64, 3, 1, 1, pool='max', norm='group', nonlin=nn.ELU),
            ConvBlock(64, 64, 3, 1, 1, pool='max', norm='group', nonlin=nn.ELU),
            make_MLP((64, 2, 2), latent_dim*2, [256, 128], nonlin=nn.ELU),
        )

        self.decoder = nn.Sequential(
            make_MLP(latent_dim, (64, 2, 2), [128, 256], nonlin=nn.ELU),
            ConvBlock(64, 64, 3, 1, 1, unpool='bilinear', norm='group', nonlin=nn.ELU),
            ConvBlock(64, 64, 3, 1, 1, unpool='bilinear', norm='group', nonlin=nn.ELU),
            ConvBlock(64, 64, 3, 1, 1, unpool='bilinear', norm='group', nonlin=nn.ELU),
            ConvBlock(64, 64, 3, 1, 1, unpool='bilinear', norm='group', nonlin=nn.ELU),
            ConvBlock(64,  3, 3, 1, 1, unpool='bilinear', norm=None, nonlin=nn.Sigmoid),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

## Train

In [ ]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# model = AutoEncoder().cuda()

model = ConvAutoEncoder.cuda()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')